In [7]:
import os,openai,backoff
import pandas as pd

dynasties= ['唐', '宋', '元']
super_powers = ['隐形', '读心术', '瞬间移动']
story_types = ['轻松']

openai.api_key = os.environ['OPENAI_API_KEY']

# new
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

# @backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def gpt35(prompt, max_tokens=2048, temperature=0.5, top_p=1, frequency_penalty=0, presence_penalty=0):
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty)
    print(response)
    return response.choices[0].text

def prepare_stories(dynasties, super_powers, story_types, output_file="data/ultraman_stories.csv"):
    df = pd.DataFrame()
    repeat = 1
    for dynasty in dynasties:
        for super_power in super_powers:
            for story_type in story_types:
                   for i in range(repeat):
                        prompt = f"""请你用中文写一段300字的故事，情节跌宕起伏，讲述一位{dynasty}朝时期的英雄人物，名字叫墨素，穿越到现代，拥有了{super_power}这样的超能力，通过{story_type}的战斗，帮助奥特曼一起打败了怪兽的故事。"""
                        story = gpt35(prompt)
                        row = {"dynasty": dynasty, "super_power": super_power, "story_type": story_type, "story": story}
                        row = pd.DataFrame([row])
                        df = pd.concat([df, row], axis=0, ignore_index=True)

    df.to_csv("data/ultraman_stories.csv")

prepare_stories(dynasties, super_powers, story_types)


Completion(id='cmpl-AHle3ajuskJiEvNkcV56VBBSoFRww', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n\n唐朝时期，有一位名叫墨素的英雄人物，他生性勇敢，心怀正义，是当时的一位有名的武将。在一次抗击外敌的战斗中，墨素不幸被敌人的毒箭射中，身负重伤。就在他即将倒下的时候，他突然发现自己穿越到了现代。\n\n墨素惊讶地发现，他拥有了隐形的超能力。他可以随意隐身，让人看不见自己。这让他感到非常兴奋，因为这样的能力可以让他更加轻松地战斗。于是，墨素决定利用这个能力，帮助现代的人们。\n\n就在他四处寻找可以帮助的地方时，他遇到了奥特曼。奥特曼是一个来自宇宙的超级英雄，他也正在与一群怪兽作战。墨素决定加入奥特曼的战斗，帮助他一起打败怪兽。\n\n墨素利用隐形的能力，轻松地躲避怪兽的攻击，并且在关键时刻，出手帮助奥特曼攻击怪兽。随着墨素的加入，奥特曼的战斗变得更加轻松，最终成功地击败了怪兽。\n\n奥特曼非常感谢墨素的帮助，他们成为了好朋友。奥特曼也向墨素介绍了现代的世界，让他了解到现代人们所面临的挑战和困难。墨素决定留在现代，继续帮助人们，并且学习现代的知识和技能。\n\n墨素的隐形能力成为了他帮助人们的最大武器。他可以在不被人们发现的情况下，解决一些犯罪问题，保护人们的安全。他也经常出现在一些危险的场合，帮助人们解决危机。\n\n随着时间的推移，墨素成为了现代人们心中的英雄。他的事迹被传颂，人们称他为“隐形英雄”。墨素也感激现代社会给予他的机会，他决心要继续用自己的能力，帮助更多的人们。\n\n墨素的故事，让人们明白，无论身处何时何地，只要心怀正义和勇气，就可以成为一位英雄。他的勇敢和善良，也将永远被人们铭记。')], created=1728798087, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=806, prompt_tokens=107, total_tokens=913, completion_token

In [8]:
df = pd.read_csv("data/ultraman_stories.csv")
df['sub_prompt'] = df['dynasty'] + "," + df['super_power'] + "," + df['story_type']
prepared_data = df.loc[:,['sub_prompt','story']]
prepared_data.rename(columns={'sub_prompt':'prompt', 'story':'completion'}, inplace=True)
prepared_data.to_csv('data/prepared_data.csv',index=False)

import subprocess

subprocess.run('openai tools fine_tunes.prepare_data --file data/prepared_data.csv --quiet'.split())


Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 9 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `,轻松`
- All completions end with suffix `。`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Necessary] Your format `CSV` will be converted to `JSONL`
- [Recommended] Add a whitespace character to the beginning of the completion [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `data/prepared_data_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:


CompletedProcess(args=['openai', 'tools', 'fine_tunes.prepare_data', '--file', 'data/prepared_data.csv', '--quiet'], returncode=0)

上传文件

openai api files.create --file data/prepared_data_prepared.jsonl --purpose fine-tune

微调

client.fine_tuning.jobs.create(
  training_file="file-abc123", 
  model="gpt-3.5-turbo"
)

查看 所有上传files

openai api files.list

In [ ]:
client.files.create(
  file=open("mydata.jsonl", "rb"),
  purpose="fine-tune"
)

In [20]:
client.fine_tuning.jobs.create(
  training_file="file-za4rxgcr3TOd5DzPagJrJY78", 
  model="davinci-002"
)

FineTuningJob(id='ftjob-gztrHnHeJWsWYezpMG9NQiBf', created_at=1728811743, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-Shug8mxv0Ze2jzMvdpj0DGHL', result_files=[], seed=328079606, status='validating_files', trained_tokens=None, training_file='file-za4rxgcr3TOd5DzPagJrJY78', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [24]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-gztrHnHeJWsWYezpMG9NQiBf")

FineTuningJob(id='ftjob-gztrHnHeJWsWYezpMG9NQiBf', created_at=1728811743, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:davinci-002:personal::AHpGlDBh', finished_at=1728812017, hyperparameters=Hyperparameters(n_epochs=9, batch_size=1, learning_rate_multiplier=16), model='davinci-002', object='fine_tuning.job', organization_id='org-Shug8mxv0Ze2jzMvdpj0DGHL', result_files=['file-JUdTHQ9EMj0kTOwKA6vJNVJV'], seed=328079606, status='succeeded', trained_tokens=76050, training_file='file-za4rxgcr3TOd5DzPagJrJY78', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [25]:
# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-gztrHnHeJWsWYezpMG9NQiBf", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-MNLzhjMkyzXZTOL32NPtj11z', created_at=1728812030, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-FcYgbrCYkVuAGZlmlge67amw', created_at=1728812029, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-OmIL4ajEarLdxvpbnwsF1qVe', created_at=1728812020, level='info', message='New fine-tuned model created', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-AZIQXW4mddGsILfAE5xYi1mR', created_at=1728812020, level='info', message='Checkpoint created at step 88', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-vq5sdbtnVcudlcvuBFyeuynU', created_at=1728812020, level='info', message='Checkpoint created at step 77', object='fine_tuning.job.event', data={

In [29]:
def gpt_fintune(prompt, max_tokens=2048, temperature=0.5, top_p=1, frequency_penalty=0, presence_penalty=0):
    response = client.completions.create(
        model="ft:davinci-002:personal::AHpGlDBh",
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty)
    print(response)
    return response.choices[0].text

In [30]:
print(gpt_fintune("五代,隐身,艰难"))

Completion(id='cmpl-AHpQe7Jko8KBkAB81KHRR4GxKbqvp', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text=',生存 \n\n墨素是五代时期的一位英雄人物，他生性勇敢，武艺高强，被人们称为“铁血战神”。他曾经在战场上立下了无数功勋，为国家保卫了边疆安宁。然而，命运却给了他一个意想不到的转折。\n\n一天，墨素突然穿越到了现代，发现自己拥有了超能力。他惊讶不已，但很快就意识到这是上天赐予他的新使命。他决定利用这个能力，继续为正义而战。\n\n就在此时，一场怪物袭击了城市，人们惊慌失措，无助地逃窜。墨素决定挺身而出，帮助奥特曼一起对抗怪物。他利用自己的超能力，轻松地躲避怪物的攻击，并且准确地预测出它的下一步动作。\n\n奥特曼感激地看着墨素，称赞他是一位真正的英雄。他们一起合力，最终将怪物打败。人们惊叹地纷纷欢呼，称墨素为“城市的新英雄”。\n\n墨素却并不自满，他知道自己的能力只是上天赐予的一次恩赐，而不是自己的全部。他决定继续努力，成为更加强大的战神。\n\n随后，墨素和奥特曼成为了好朋友，一起保卫着城市的安全。他们经常一起出击，对抗各种恶势力。墨素的战斗技巧得到了进一步的提高，他能够与奥特曼并驾齐驱，击败更加强大的怪物。\n\n随着时间的推移，墨素逐渐融入了现代社会，他学习了现代科技，帮助奥特曼改良装备，使他们的战斗更加灵活强大。\n\n然而，越来越多的怪物开始出现，它们的力量也越来越强大。奥特曼和墨素面临着前所未有的挑战。\n\n为了保卫城市的和平，奥特曼和墨素不断地磨练自己的技能，最终在一次激烈的战斗中，将最强大的怪物击败。\n\n人们惊叹地纷纷欢呼，称墨素和奥特曼为“城市的最强英雄”。\n\n墨素感激地看着人们，他知道自己的力量来自上天，而不是自己的全部。他选择了保护人民，而不是自己的私利，成为了一位真正的英雄。\n\n墨素和奥特曼的友谊也变得更加深厚，他们一起回忆起过去的战斗，分享彼此的故事。\n\n墨素最后选择留下来，成为了现代社会中的一位传奇人物。他的英勇事迹被后人传颂，他的名字被铭记在人们的心中，永远不会被遗忘。\n\n墨素用自己的力量，继续保卫着正义和和平，成为了一代又一代人心中的英雄。他的故事将永远